# Decision Trees
In this notebook, we will be using the Decision Trees in order to try and predict individual games. We will be using the data that was collected from the sportsreference API from the summer of 2020. Models will be trained on the moving averages that we generated.

In [88]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

from tqdm import tqdm
from sklearn.model_selection import train_test_split

pd.set_option("display.max_rows", None, "display.max_columns", None)

# Getting Data Ready

In [89]:
# Lead data.csv
data_ma = pd.read_csv('../assets/data/cleaned_data/cleaned_data.csv')
data_ma.head()

,date,location,home,away,winner,home_assist_percentage_SMA,home_assist_percentage_CMA,home_assist_percentage_EMA,home_assists_SMA,home_assists_CMA,home_assists_EMA,home_block_percentage_SMA,home_block_percentage_CMA,home_block_percentage_EMA,home_blocks_SMA,home_blocks_CMA,home_blocks_EMA,home_defensive_rating_SMA,home_defensive_rating_CMA,home_defensive_rating_EMA,home_defensive_rebound_percentage_SMA,home_defensive_rebound_percentage_CMA,home_defensive_rebound_percentage_EMA,home_defensive_rebounds_SMA,home_defensive_rebounds_CMA,home_defensive_rebounds_EMA,home_effective_field_goal_percentage_SMA,home_effective_field_goal_percentage_CMA,home_effective_field_goal_percentage_EMA,home_field_goal_attempts_SMA,home_field_goal_attempts_CMA,home_field_goal_attempts_EMA,home_field_goal_percentage_SMA,home_field_goal_percentage_CMA,home_field_goal_percentage_EMA,home_field_goals_SMA,home_field_goals_CMA,home_field_goals_EMA,home_free_throw_attempt_rate_SMA,home_free_throw_attempt_rate_CMA,home_free_throw_attempt_rate_EMA,home_free_throw_attempts_SMA,home_free_throw_attempts_CMA,home_free_throw_attempts_EMA,home_free_throw_percentage_SMA,home_free_throw_percentage_CMA,home_free_throw_percentage_EMA,home_free_throws_SMA,home_free_throws_CMA,home_free_throws_EMA,home_losses_SMA,home_losses_CMA,home_losses_EMA,home_minutes_played_SMA,home_minutes_played_CMA,home_minutes_played_EMA,home_offensive_rating_SMA,home_offensive_rating_CMA,home_offensive_rating_EMA,home_offensive_rebound_percentage_SMA,home_offensive_rebound_percentage_CMA,home_offensive_rebound_percentage_EMA,home_offensive_rebounds_SMA,home_offensive_rebounds_CMA,home_offensive_rebounds_EMA,home_personal_fouls_SMA,home_personal_fouls_CMA,home_personal_fouls_EMA,home_points_SMA,home_points_CMA,home_points_EMA,home_steal_percentage_SMA,home_steal_percentage_CMA,home_steal_percentage_EMA,home_steals_SMA,home_steals_CMA,home_steals_EMA,home_three_point_attempt_rate_SMA,home_three_point_attempt_rate_CMA,home_three_point_attempt_rate_EMA,home_three_point_field_goal_attempts_SMA,home_three_point_field_goal_attempts_CMA,home_three_point_field_goal_attempts_EMA,home_three_point_field_goal_percentage_SMA,home_three_point_field_goal_percentage_CMA,home_three_point_field_goal_percentage_EMA,home_three_point_field_goals_SMA,home_three_point_field_goals_CMA,home_three_point_field_goals_EMA,home_total_rebound_percentage_SMA,home_total_rebound_percentage_CMA,home_total_rebound_percentage_EMA,home_total_rebounds_SMA,home_total_rebounds_CMA,home_total_rebounds_EMA,home_true_shooting_percentage_SMA,home_true_shooting_percentage_CMA,home_true_shooting_percentage_EMA,home_turnover_percentage_SMA,home_turnover_percentage_CMA,home_turnover_percentage_EMA,home_turnovers_SMA,home_turnovers_CMA,home_turnovers_EMA,home_two_point_field_goal_attempts_SMA,home_two_point_field_goal_attempts_CMA,home_two_point_field_goal_attempts_EMA,home_two_point_field_goal_percentage_SMA,home_two_point_field_goal_percentage_CMA,home_two_point_field_goal_percentage_EMA,home_two_point_field_goals_SMA,home_two_point_field_goals_CMA,home_two_point_field_goals_EMA,home_win_percentage_SMA,home_win_percentage_CMA,home_win_percentage_EMA,home_wins_SMA,home_wins_CMA,home_wins_EMA,away_assist_percentage_SMA,away_assist_percentage_CMA,away_assist_percentage_EMA,away_assists_SMA,away_assists_CMA,away_assists_EMA,away_block_percentage_SMA,away_block_percentage_CMA,away_block_percentage_EMA,away_blocks_SMA,away_blocks_CMA,away_blocks_EMA,away_defensive_rating_SMA,away_defensive_rating_CMA,away_defensive_rating_EMA,away_defensive_rebound_percentage_SMA,away_defensive_rebound_percentage_CMA,away_defensive_rebound_percentage_EMA,away_defensive_rebounds_SMA,away_defensive_rebounds_CMA,away_defensive_rebounds_EMA,away_effective_field_goal_percentage_SMA,away_effective_field_goal_percentage_CMA,away_effective_field_goal_percentage_EMA,away_field_goal_attempts_SMA,away_field_goal_attempts_CMA,away_field_goal_attempts_EMA,away_field_goal_perc

In [90]:
# These fields aren't necessary to train the model
DROPPING = ['date',
            'location',
            'home',
            'away']

ma = data_ma.copy()
ma['target'] = np.where(ma['winner'] == "Home", 0, 1)
ma.drop(columns=DROPPING, inplace=True)
ma.dropna(inplace=True)
ma.drop_duplicates(inplace=True)
ma.pop('winner')

target = ma.pop('target')
ma = ma.filter(regex="EMA", axis=1)
ma['target'] = target
ma.head()

,home_assist_percentage_EMA,home_assists_EMA,home_block_percentage_EMA,home_blocks_EMA,home_defensive_rating_EMA,home_defensive_rebound_percentage_EMA,home_defensive_rebounds_EMA,home_effective_field_goal_percentage_EMA,home_field_goal_attempts_EMA,home_field_goal_percentage_EMA,home_field_goals_EMA,home_free_throw_attempt_rate_EMA,home_free_throw_attempts_EMA,home_free_throw_percentage_EMA,home_free_throws_EMA,home_losses_EMA,home_minutes_played_EMA,home_offensive_rating_EMA,home_offensive_rebound_percentage_EMA,home_offensive_rebounds_EMA,home_personal_fouls_EMA,home_points_EMA,home_steal_percentage_EMA,home_steals_EMA,home_three_point_attempt_rate_EMA,home_three_point_field_goal_attempts_EMA,home_three_point_field_goal_percentage_EMA,home_three_point_field_goals_EMA,home_total_rebound_percentage_EMA,home_total_rebounds_EMA,home_true_shooting_percentage_EMA,home_turnover_percentage_EMA,home_turnovers_EMA,home_two_point_field_goal_attempts_EMA,home_two_point_field_goal_percentage_EMA,home_two_point_field_goals_EMA,home_win_percentage_EMA,home_wins_EMA,away_assist_percentage_EMA,away_assists_EMA,away_block_percentage_EMA,away_blocks_EMA,away_defensive_rating_EMA,away_defensive_rebound_percentage_EMA,away_defensive_rebounds_EMA,away_effective_field_goal_percentage_EMA,away_field_goal_attempts_EMA,away_field_goal_percentage_EMA,away_field_goals_EMA,away_free_throw_attempt_rate_EMA,away_free_throw_attempts_EMA,away_free_throw_percentage_EMA,away_free_throws_EMA,away_losses_EMA,away_minutes_played_EMA,away_offensive_rating_EMA,away_offensive_rebound_percentage_EMA,away_offensive_rebounds_EMA,away_personal_fouls_EMA,away_points_EMA,away_steal_percentage_EMA,away_steals_EMA,away_three_point_attempt_rate_EMA,away_three_point_field_goal_attempts_EMA,away_three_point_field_goal_percentage_EMA,away_three_point_field_goals_EMA,away_total_rebound_percentage_EMA,away_total_rebounds_EMA,away_true_shooting_percentage_EMA,away_turnover_percentage_EMA,away_turnovers_EMA,away_two_point_field_goal_attempts_EMA,away_two_point_field_goal_percentage_EMA,away_two_point_field_goals_EMA,away_win_percentage_EMA,away_wins_EMA,target
0,62.004938,15.037037,9.353086,4.370370,105.667901,53.988889,21.580247,0.542457,52.012346,0.468840,24.308642,0.336951,17.111111,0.716889,13.012346,2.061728,200.0,106.697531,32.645679,7.728395,15.197531,69.197531,5.653086,3.790123,0.404148,21.271605,0.360296,7.567901,44.724691,29.308642,0.576741,19.002469,14.160494,30.740741,0.546012,16.740741,0.485136,1.333333,59.146914,16.419753,8.658025,3.382716,91.364198,71.012346,24.604938,0.511000,59.296296,0.478296,27.827160,0.439531,25.296296,0.635605,16.074074,1.037037,200.0,101.174074,41.734568,12.259259,16.851852,75.506173,12.179012,9.086420,0.242407,14.617284,0.273926,3.777778,55.933333,36.864198,0.534086,19.382716,16.629630,44.679012,0.542716,24.049383,0.761778,2.358025,1
1,52.204938,9.925926,3.781481,1.320988,121.093827,63.783951,19.827160,0.406852,52.728395,0.360926,18.962963,0.451420,23.530864,0.634691,14.358025,3.395062,200.0,86.409877,31.037037,9.432099,18.592593,57.037037,5.969136,3.975309,0.328235,17.345679,0.288222,4.753086,45.819753,29.259259,0.447160,17.402469,13.123457,35.382716,0.403074,14.209877,0.000000,0.000000,64.554321,14.086420,9.627160,4.234568,100.969136,70.982716,26.123457,0.466395,52.246914,0.419284,21.814815,0.346432,17.913580,0.650889,10.851852,4.395062,200.0,87.774074,33.150617,8.530864,16.938272,59.333333,4.992593,3.296296,0.267222,13.888889,0.348457,4.851852,53.043210,34.654321,0.488914,22.316049,17.395062,38.358025,0.442778,16.962963,0.000000,0.000000,0
2,54.185185,12.925926,5.856790,1.765432,109.480247,56.780247,18.345679,0.480593,56.518519,0.421605,23.790123,0.271901,15.308642,0.672556,10.382716,3.061728,200.0,93.345679,45.044444,12.012346,20.271605,64.555556,14.229630,10.012346,0.325037,18.382716,0.360346,6.592593,48.972840,30.358025,0.506667,23.519753,18.790123,38.135802,0.450247,17.197531,0.066667,0.333333,52.893827,10.827160,5.544444,2.000000,101

In [91]:
# create training and test samples
train_ma, test_ma = train_test_split(ma, test_size=0.2)
print(len(train_ma), 'train examples')
print(len(test_ma), 'test examples')

# extract training and test labels
train_true = train_ma.pop('target')
test_true = test_ma.pop('target')

32588 train examples
8147 test examples


# Random Forest Classifier

In [125]:
parameters = {'verbose':0,
              'n_jobs':-1,
              'bootstrap': False,
              'min_samples_leaf': 3,
              'n_estimators': 50,
              'min_samples_split': 10,
              'max_features': 'sqrt',
              'max_depth': 20}
clf = RandomForestClassifier(**parameters)
clf.fit(train_ma, train_true)
accuracy = clf.score(test_ma, test_true)
print(f'Accuracy of model: {accuracy*100:.2f}%')

Accuracy of model: 70.13%


# Gradient Boosting Classifier

In [137]:
parameters = {'verbose':0,
              'loss': 'exponential',
              'n_estimators':100,
              'min_samples_leaf': 3,
              'min_samples_split': 10,
              'max_features': 'sqrt',
              'max_depth': 20}
clf = GradientBoostingClassifier(**parameters)
clf.fit(train_ma, train_true)
accuracy = clf.score(test_ma, test_true)
print(f'Accuracy of model: {accuracy*100:.2f}%')

Accuracy of model: 70.55%


# Analysis
As seen above, we were able to surpass 70%, which is quite impressive when predicting sports models. I believe we will be able to increase accuracy when we retrain these models on a larger dataset. Stay tuned when we retrain on our 2010-2020 dataset versus our current 2020 dataset.

In [12]:
from joblib import dump, load

parameters = {'verbose':0,
              'n_jobs':-1,
              'bootstrap': False,
              'min_samples_leaf': 3,
              'n_estimators': 50,
              'min_samples_split': 10,
              'max_features': 'sqrt',
              'max_depth': 20}

accuracy = 0
while accuracy < .7:
    clf = RandomForestClassifier(**parameters)
    clf.fit(train_ma, train_true)
    accuracy = clf.score(test_ma, test_true)
    print(f'Accuracy of model: {accuracy*100:.2f}%')

dump(clf, 'rfc.joblib') 

Accuracy of model: 68.56%
Accuracy of model: 69.84%
Accuracy of model: 70.13%


['rfc.joblib']

In [14]:
clf.predict_proba(np.array([[ 50.18853319,  12.83365005,   7.39975925,   2.8530138 ,
        103.21410579,  69.34915761,  21.54884196,   0.51349807,
         56.43708545,   0.4528056 ,  25.50194843,   0.33992502,
         19.12640233,   0.66247622,  12.5104348 ,  10.6712573 ,
        200.28978332, 102.28501627,  33.95708128,  10.89455019,
         14.32178333,  70.38661533,  11.33309182,   7.81303643,
          0.36325609,  20.55728647,   0.33363987,   6.87228367,
         51.39635693,  32.44339216,   0.53815396,  18.28670799,
         14.67612811,  35.87979898,   0.52098572,  18.62966476,
          0.63308065,  18.32875313,  41.66260908,   9.4091232 ,
         10.98609931,   3.67935434, 111.15390629,  67.04092165,
         21.07336026,   0.40108225,  59.40800058,   0.36711136,
         21.88518278,   0.30188099,  17.64448853,   0.63085532,
         12.46386906,  14.5807768 , 200.02966541,  85.99923067,
         31.58889834,  11.65310409,  18.82367792,  60.30295817,
          7.63371439,   5.3567371 ,   0.29073411,  17.23424975,
          0.23374772,   4.06872355,  47.92145449,  32.72646435,
          0.44322548,  16.99650986,  13.79067675,  42.17375082,
          0.42040243,  17.81645923,   0.49954837,  14.41923363]]))

array([[0.32494444, 0.67505556]])

In [15]:
from joblib import dump, load

parameters = {'verbose':0,
              'loss': 'exponential',
              'n_estimators':100,
              'min_samples_leaf': 3,
              'min_samples_split': 10,
              'max_features': 'sqrt',
              'max_depth': 20}

accuracy = 0
while accuracy < .7:
    clf = GradientBoostingClassifier(**parameters)
    clf.fit(train_ma, train_true)
    accuracy = clf.score(test_ma, test_true)
    print(f'Accuracy of model: {accuracy*100:.2f}%')

dump(clf, 'rfc.joblib') 

Accuracy of model: 68.14%
Accuracy of model: 69.56%
Accuracy of model: 71.12%


['rfc.joblib']

In [16]:
clf.predict_proba(np.array([[ 50.18853319,  12.83365005,   7.39975925,   2.8530138 ,
        103.21410579,  69.34915761,  21.54884196,   0.51349807,
         56.43708545,   0.4528056 ,  25.50194843,   0.33992502,
         19.12640233,   0.66247622,  12.5104348 ,  10.6712573 ,
        200.28978332, 102.28501627,  33.95708128,  10.89455019,
         14.32178333,  70.38661533,  11.33309182,   7.81303643,
          0.36325609,  20.55728647,   0.33363987,   6.87228367,
         51.39635693,  32.44339216,   0.53815396,  18.28670799,
         14.67612811,  35.87979898,   0.52098572,  18.62966476,
          0.63308065,  18.32875313,  41.66260908,   9.4091232 ,
         10.98609931,   3.67935434, 111.15390629,  67.04092165,
         21.07336026,   0.40108225,  59.40800058,   0.36711136,
         21.88518278,   0.30188099,  17.64448853,   0.63085532,
         12.46386906,  14.5807768 , 200.02966541,  85.99923067,
         31.58889834,  11.65310409,  18.82367792,  60.30295817,
          7.63371439,   5.3567371 ,   0.29073411,  17.23424975,
          0.23374772,   4.06872355,  47.92145449,  32.72646435,
          0.44322548,  16.99650986,  13.79067675,  42.17375082,
          0.42040243,  17.81645923,   0.49954837,  14.41923363]]))

array([[3.21587005e-06, 9.99996784e-01]])

In [4]:
from sportsipy.ncaab.teams import Teams

for team in Teams(2021):
    print(team, team.name, team.abbreviation)

Abilene Christian (ABILENE-CHRISTIAN) - 2021 Abilene Christian ABILENE-CHRISTIAN
Air Force (AIR-FORCE) - 2021 Air Force AIR-FORCE
Akron (AKRON) - 2021 Akron AKRON
Alabama A&M (ALABAMA-AM) - 2021 Alabama A&M ALABAMA-AM
Alabama-Birmingham (ALABAMA-BIRMINGHAM) - 2021 Alabama-Birmingham ALABAMA-BIRMINGHAM
Alabama State (ALABAMA-STATE) - 2021 Alabama State ALABAMA-STATE
Alabama (ALABAMA) - 2021 Alabama ALABAMA
Albany (NY) (ALBANY-NY) - 2021 Albany (NY) ALBANY-NY
Alcorn State (ALCORN-STATE) - 2021 Alcorn State ALCORN-STATE
American (AMERICAN) - 2021 American AMERICAN
Appalachian State (APPALACHIAN-STATE) - 2021 Appalachian State APPALACHIAN-STATE
Arizona State (ARIZONA-STATE) - 2021 Arizona State ARIZONA-STATE
Arizona (ARIZONA) - 2021 Arizona ARIZONA
Little Rock (ARKANSAS-LITTLE-ROCK) - 2021 Little Rock ARKANSAS-LITTLE-ROCK
Arkansas-Pine Bluff (ARKANSAS-PINE-BLUFF) - 2021 Arkansas-Pine Bluff ARKANSAS-PINE-BLUFF
Arkansas State (ARKANSAS-STATE) - 2021 Arkansas State ARKANSAS-STATE
Arkansas (AR

In [5]:
ma = {}
for team in Teams(2021):
    try:
        dataset = pd.read_csv(f'./team_ma/{team.abbreviation}_ma.csv')
        dataset = dataset.tail(1)
        dataset = dataset.filter(regex="EMA", axis=1)
        # dataset.drop(columns=['date', 'location', 'away', 'home', 'winner'], inplace=True)
        # print(dataset.values.tolist()[0])
        ma[team.name] = dataset.values.tolist()[0]
    except:
        print(f'Did not find: {team.abbreviation}')

In [34]:
dataset = pd.read_csv(f'./team_ma/CONNECTICUT_ma.csv')
dataset = dataset.tail(1)
dataset = dataset.filter(regex="EMA", axis=1)
dataset.drop(columns=['date', 'location', 'away', 'home', 'winner'], inplace=True)
dataset.style

KeyError: "['date' 'location' 'away' 'home' 'winner'] not found in axis"

In [6]:
ma

{'Abilene Christian': [68.78546170419655,
  20.52530945192172,
  8.331377903428756,
  2.775699349433268,
  91.83629680864608,
  80.87978618442426,
  22.952510332696143,
  0.5627146295522558,
  61.01895555610673,
  0.4887032818523159,
  29.77609612326477,
  0.349583996101608,
  20.464160348009937,
  0.7516310843902458,
  15.041659142894112,
  3.796619583204828,
  200.00000763090983,
  114.81567149013934,
  32.402133971478335,
  10.03123753106822,
  18.38624777405365,
  83.81996811109289,
  13.906547238831209,
  10.22419350403314,
  0.3706343661802985,
  22.946347630777765,
  0.4025295242491961,
  9.226116721669223,
  55.64463389793092,
  32.983747863764364,
  0.5913901865606139,
  15.90625256236792,
  13.173948744759066,
  38.07260792532895,
  0.5399867843678758,
  20.54997940159555,
  0.8415347704622544,
  20.20427806590112],
 'Air Force': [57.001019929392015,
  10.613583654369862,
  10.89080740888036,
  2.882016663266901,
  110.50873744389088,
  63.240213847602206,
  16.76910399766253

In [7]:
import json
with open("ma.json", "w") as write_file:
    json.dump(ma, write_file, sort_keys=True, indent=4)

In [92]:
from joblib import dump, load

lr_clf = load('./models/logisticreg.joblib')
svm_clf = load('./models/svm.joblib')
rfc_clf = load('./models/rfc.joblib')
gbc_clf = load('./models/gbc.joblib')

In [93]:
labels = ma.pop('target')

In [94]:
ma.head()

,home_assist_percentage_EMA,home_assists_EMA,home_block_percentage_EMA,home_blocks_EMA,home_defensive_rating_EMA,home_defensive_rebound_percentage_EMA,home_defensive_rebounds_EMA,home_effective_field_goal_percentage_EMA,home_field_goal_attempts_EMA,home_field_goal_percentage_EMA,home_field_goals_EMA,home_free_throw_attempt_rate_EMA,home_free_throw_attempts_EMA,home_free_throw_percentage_EMA,home_free_throws_EMA,home_losses_EMA,home_minutes_played_EMA,home_offensive_rating_EMA,home_offensive_rebound_percentage_EMA,home_offensive_rebounds_EMA,home_personal_fouls_EMA,home_points_EMA,home_steal_percentage_EMA,home_steals_EMA,home_three_point_attempt_rate_EMA,home_three_point_field_goal_attempts_EMA,home_three_point_field_goal_percentage_EMA,home_three_point_field_goals_EMA,home_total_rebound_percentage_EMA,home_total_rebounds_EMA,home_true_shooting_percentage_EMA,home_turnover_percentage_EMA,home_turnovers_EMA,home_two_point_field_goal_attempts_EMA,home_two_point_field_goal_percentage_EMA,home_two_point_field_goals_EMA,home_win_percentage_EMA,home_wins_EMA,away_assist_percentage_EMA,away_assists_EMA,away_block_percentage_EMA,away_blocks_EMA,away_defensive_rating_EMA,away_defensive_rebound_percentage_EMA,away_defensive_rebounds_EMA,away_effective_field_goal_percentage_EMA,away_field_goal_attempts_EMA,away_field_goal_percentage_EMA,away_field_goals_EMA,away_free_throw_attempt_rate_EMA,away_free_throw_attempts_EMA,away_free_throw_percentage_EMA,away_free_throws_EMA,away_losses_EMA,away_minutes_played_EMA,away_offensive_rating_EMA,away_offensive_rebound_percentage_EMA,away_offensive_rebounds_EMA,away_personal_fouls_EMA,away_points_EMA,away_steal_percentage_EMA,away_steals_EMA,away_three_point_attempt_rate_EMA,away_three_point_field_goal_attempts_EMA,away_three_point_field_goal_percentage_EMA,away_three_point_field_goals_EMA,away_total_rebound_percentage_EMA,away_total_rebounds_EMA,away_true_shooting_percentage_EMA,away_turnover_percentage_EMA,away_turnovers_EMA,away_two_point_field_goal_attempts_EMA,away_two_point_field_goal_percentage_EMA,away_two_point_field_goals_EMA,away_win_percentage_EMA,away_wins_EMA
0,62.004938,15.037037,9.353086,4.370370,105.667901,53.988889,21.580247,0.542457,52.012346,0.468840,24.308642,0.336951,17.111111,0.716889,13.012346,2.061728,200.0,106.697531,32.645679,7.728395,15.197531,69.197531,5.653086,3.790123,0.404148,21.271605,0.360296,7.567901,44.724691,29.308642,0.576741,19.002469,14.160494,30.740741,0.546012,16.740741,0.485136,1.333333,59.146914,16.419753,8.658025,3.382716,91.364198,71.012346,24.604938,0.511000,59.296296,0.478296,27.827160,0.439531,25.296296,0.635605,16.074074,1.037037,200.0,101.174074,41.734568,12.259259,16.851852,75.506173,12.179012,9.086420,0.242407,14.617284,0.273926,3.777778,55.933333,36.864198,0.534086,19.382716,16.629630,44.679012,0.542716,24.049383,0.761778,2.358025
1,52.204938,9.925926,3.781481,1.320988,121.093827,63.783951,19.827160,0.406852,52.728395,0.360926,18.962963,0.451420,23.530864,0.634691,14.358025,3.395062,200.0,86.409877,31.037037,9.432099,18.592593,57.037037,5.969136,3.975309,0.328235,17.345679,0.288222,4.753086,45.819753,29.259259,0.447160,17.402469,13.123457,35.382716,0.403074,14.209877,0.000000,0.000000,64.554321,14.086420,9.627160,4.234568,100.969136,70.982716,26.123457,0.466395,52.246914,0.419284,21.814815,0.346432,17.913580,0.650889,10.851852,4.395062,200.0,87.774074,33.150617,8.530864,16.938272,59.333333,4.992593,3.296296,0.267222,13.888889,0.348457,4.851852,53.043210,34.654321,0.488914,22.316049,17.395062,38.358025,0.442778,16.962963,0.000000,0.000000
2,54.185185,12.925926,5.856790,1.765432,109.480247,56.780247,18.345679,0.480593,56.518519,0.421605,23.790123,0.271901,15.308642,0.672556,10.382716,3.061728,200.0,93.345679,45.044444,12.012346,20.271605,64.555556,14.229630,10.012346,0.325037,18.382716,0.360346,6.592593,48.972840,30.358025,0.506667,23.519753,18.790123,38.135802,0.450247,17.197531,0.066667,0.333333,52.893827,10.827160,5.544444,2.000000,101.125926,65.

In [95]:
avg = np.mean( np.array([lr_clf.predict_proba(ma), svm_clf.predict_proba(ma), rfc_clf.predict_proba(ma), gbc_clf.predict_proba(ma)]), axis=0 )


In [96]:
print(avg)

[[0.69136992 0.30863008]
 [0.45840657 0.54159343]
 [0.57792409 0.42207591]
 [0.79473303 0.20526697]
 [0.70411229 0.29588771]
 [0.93995628 0.06004372]
 [0.58034748 0.41965252]
 [0.84655828 0.15344172]
 [0.73731182 0.26268818]
 [0.88170943 0.11829057]
 [0.41531759 0.58468241]
 [0.71231145 0.28768855]
 [0.65130088 0.34869912]
 [0.69611306 0.30388694]
 [0.55837433 0.44162567]
 [0.58815313 0.41184687]
 [0.45857805 0.54142195]
 [0.75549879 0.24450121]
 [0.67411635 0.32588365]
 [0.63023672 0.36976328]
 [0.89215014 0.10784986]
 [0.57401047 0.42598953]
 [0.92037322 0.07962678]
 [0.81214963 0.18785037]
 [0.51719614 0.48280386]
 [0.54461393 0.45538607]
 [0.72300405 0.27699595]
 [0.84906199 0.15093801]
 [0.76367137 0.23632863]
 [0.44980458 0.55019542]
 [0.7008096  0.2991904 ]
 [0.92886493 0.07113507]
 [0.87453012 0.12546988]
 [0.65435623 0.34564377]
 [0.77964581 0.22035419]
 [0.76470209 0.23529791]
 [0.77654114 0.22345886]
 [0.86471005 0.13528995]
 [0.94199875 0.05800125]
 [0.86582868 0.13417132]


In [97]:
avg[:,1].round()

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
       0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1.,
       0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       1., 0., 0., 0., 0.

In [98]:
accuracy_score(avg[:,1].round(), labels)

0.6983429483245367

In [68]:
train_true.values

array([1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,

In [20]:
lr_clf.predict_proba(ma)

array([[0.60826199, 0.39173801],
       [0.88277012, 0.11722988],
       [0.37067934, 0.62932066],
       ...,
       [0.39907044, 0.60092956],
       [0.60932703, 0.39067297],
       [0.88546   , 0.11454   ]])

In [22]:
svm_clf.predict_proba(ma)

array([[0.61266827, 0.38733173],
       [0.85593622, 0.14406378],
       [0.5       , 0.5       ],
       ...,
       [0.50620377, 0.49379623],
       [0.59013673, 0.40986327],
       [0.79903225, 0.20096775]])

In [23]:
rfc_clf.predict_proba(ma)

array([[0.19294375, 0.80705625],
       [0.95109524, 0.04890476],
       [0.83838095, 0.16161905],
       ...,
       [0.13653175, 0.86346825],
       [0.81397863, 0.18602137],
       [0.32494444, 0.67505556]])

In [24]:
gbc_clf.predict_proba(ma)

array([[2.67891165e-06, 9.99997321e-01],
       [9.99999815e-01, 1.85123324e-07],
       [9.99998667e-01, 1.33275534e-06],
       ...,
       [1.46475906e-06, 9.99998535e-01],
       [9.99999180e-01, 8.20303562e-07],
       [3.21587005e-06, 9.99996784e-01]])

In [55]:
ma.values[:,0]

array([60.74814815, 47.31646091, 58.08641975, ..., 37.75432806,
       50.78054505, 50.18853319])

In [87]:
c = np.array([[0.86962275, 0.13037725]])